In [1]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.2 MB/s eta 0:00:0000:01


In [2]:
# !pip install py_vncorenlp

In [2]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import pandas as pd
from sentence_transformers import util

In [4]:
# import py_vncorenlp
# rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/input/vncorenlp/VnCoreNLP-master')

# Models

In [3]:
from sentence_transformers import SentenceTransformer
import torch

# Kiểm tra xem GPU có sẵn không
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Tải mô hình và chỉ định thiết bị
model_path = "/kaggle/input/m/ictunivers/halong_embedding/transformers/default/1/checkpoint-2500"
model = SentenceTransformer(model_path, device=device)

# Kiểm tra xem model đã được tải trên GPU hay không
print(f"Model device: {model.device}")


Using device: cuda
Model device: cuda:0


In [4]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [5]:
from sentence_transformers import CrossEncoder
import torch

# Kiểm tra xem GPU có sẵn không
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Tải mô hình và chỉ định thiết bị
model_name = "BAAI/bge-reranker-v2-m3"
cross_encoder = CrossEncoder(model_name, num_labels=1, max_length=1024, device=device)


Using device: cuda


config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [8]:
# model_name = "/kaggle/input/phoranker_finetune_jan/transformers/default/1/phoranker_finetune_Dec"
# cross_encoder = CrossEncoder(model_name, num_labels=1, max_length=256)

In [6]:
import logging
logging.disable(logging.WARNING)

In [7]:
model_cross = cross_encoder.model

# Print the model architecture
print(model_cross)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(8194, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_f

# Corpus

In [9]:
corpus = pd.read_csv("/kaggle/input/healthcare/Corpus (1).csv")

In [10]:
print(corpus.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25668 entries, 0 to 25667
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   corpus  25668 non-null  object
dtypes: object(1)
memory usage: 200.7+ KB
None


In [13]:
# import pandas as pd
# import h5py
# from tqdm import tqdm
# import numpy as np


# # Extract texts 
# texts = corpus["context"].tolist() 
# cids = corpus['cid'].tolist()


# batch_size = 128  # Adjust this based on your system's memory
# embeddings = []
# model = model.to('cuda')
# print(model.device)
# print("Encoding text...")
# for start_idx in tqdm(range(0, len(texts), batch_size)):
#     batch_texts = texts[start_idx:start_idx + batch_size]
#     batch_embeddings = model.encode(batch_texts, convert_to_numpy=True, show_progress_bar=False)
#     embeddings.append(batch_embeddings)

# embeddings = np.vstack(embeddings)
# print("start embeddings")
# output_file = "/kaggle/working/corpus_embeddings_halong_03.h5"  
# print(f"Saving embeddings to {output_file}...")
# with h5py.File(output_file, "w") as f:
#     f.create_dataset("embeddings", data=embeddings)
#     f.create_dataset("cids", data=np.array(cids, dtype="S")) 

# print("Embeddings saved successfully.")

# Vector embeddings call 

In [11]:
import h5py

# Đường dẫn đến tệp HDF5
file_path = "/kaggle/input/healthcare/doc_embeddings.pt"

# Mở tệp HDF5
with h5py.File(file_path, "r") as f:
    # Hiển thị các nhóm trong tệp
    print("Các nhóm trong tệp HDF5:", list(f.keys()))

    # Lấy nhóm cids và embeddings
    cids = f['cids'][:]  # Lấy toàn bộ dữ liệu trong nhóm cids
    embeddings = f['embeddings'][:]  # Lấy toàn bộ dữ liệu trong nhóm embeddings

    # Lấy một mẫu từ nhóm cids và nhóm embeddings
    cid_sample = cids[0]  # Ví dụ: lấy mẫu đầu tiên từ cids
    embedding_sample = embeddings[0]  # Lấy mẫu đầu tiên từ embeddings

    # Hiển thị kết quả
    print("Sample từ nhóm cids:", cid_sample)  # Giải mã chuỗi byte
    print("Sample từ nhóm embeddings:", embedding_sample)

OSError: Unable to synchronously open file (file signature not found)

In [13]:
import torch

# Đường dẫn đến tệp .pt
file_path = "/kaggle/input/healthcare/doc_embeddings.pt"

# Tải dữ liệu
data = torch.load(file_path)

# Kiểm tra và xử lý dữ liệu
if isinstance(data, torch.Tensor):
    # Nếu toàn bộ file là một tensor
    embeddings = data
elif isinstance(data, dict) and 'embeddings' in data:
    # Nếu file là dictionary và chứa key 'embeddings'
    embeddings = data['embeddings']
else:
    raise ValueError("File không chứa embeddings hoặc không đúng định dạng!")

# Hiển thị thông tin embeddings
print(f"Tổng số embeddings: {len(embeddings)}")
print("Embedding mẫu:", embeddings[0])

# Đảm bảo kiểu dữ liệu là tensor
if isinstance(embeddings, torch.Tensor):
    print(f"Kích thước của embedding: {embeddings.shape}")


Tổng số embeddings: 25668
Embedding mẫu: tensor([-2.5561e-01, -2.2460e-01,  2.7973e-01,  2.9836e-01,  5.6402e-03,
         3.0811e-01, -9.0863e-03,  2.9652e-01, -5.1866e-02,  3.6566e-01,
        -1.5304e-01,  7.3074e-02, -2.1834e-02,  4.5647e-04,  1.7569e-01,
        -1.6430e-01, -4.3231e-02, -2.1705e-01, -1.1724e-01, -3.0267e-01,
         2.7409e-01, -8.0533e-02, -1.3059e-01, -2.8147e-01,  6.0854e-01,
        -7.5169e-02,  2.4090e-01, -1.1425e-01,  6.9873e-02, -9.1570e-03,
        -4.5452e-02,  1.6003e-01,  3.3195e-02, -1.0014e-01, -5.3177e-01,
         8.1638e-02,  2.9084e-02, -9.9377e-02,  1.2253e-01, -7.3335e-01,
         7.0233e-02, -1.0116e-01,  1.6906e-01,  1.0135e-01, -1.1252e-01,
         1.0180e-01, -1.0976e-01, -5.1516e-01, -3.3952e-01,  1.5647e-01,
         7.7569e-02,  8.1755e-02,  1.4685e-01,  4.1210e-01,  3.0738e-01,
        -2.3537e-01,  8.5673e-02,  6.0791e-02, -2.7032e-01,  3.8287e-02,
         3.9097e-01,  1.3501e-01,  3.9549e-02, -5.2846e-02, -4.2594e-01,
         3

<ipython-input-13-5240e0dc9989>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)


In [14]:
import re

def preprocess_text(text):
    # Replace newline characters with spaces
    text = ' '.join(text.splitlines())
    
    # Replace escaped newline characters with periods
    text = text.replace('\\n', '.')
    
    # Remove specific unwanted characters and patterns
    text = text.replace('/', '')
    text = text.replace("''", '')
    text = text.strip("[]")
    text = text.strip("'")
    text = text.strip('"')
    
    # Remove numbered list indicators and lettered list indicators
    text = re.sub(r'\b\d\.\s*', '', text)  # Removes patterns like '1. ', '2. ', etc.
    text = re.sub(r'\b[a-e]\)\s*', '', text)  # Removes patterns like 'a) ', 'b) ', etc.
    
    # Replace certain punctuation with periods
    text = text.replace(';', '.')
    text = text.replace(':', '.')
    
    # Remove hyphens
    text = text.replace('-', '')
    
    # Strip leading and trailing whitespace
    text = text.strip()

    text = re.sub(r'\s{2,}', ' ', text)

    
    return text



In [16]:
import logging
logging.disable(logging.WARNING)

# Chunking for re-ranking

In [17]:
from nltk.tokenize import sent_tokenize

In [18]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# Define the input text
text = """BỘT CÁNH - CẲNG - BÀN TAY\n...\n2. Phương tiện\n- 01 bàn nắn thông thường, tốt nhất như kiểu bàn mổ (chắc, nặng, để khi kéo nắn bàn không bị chạy). Ở nơi không có điều kiện, có thể dùng bàn sắt, bàn gỗ, nhưng chân bàn phải được cố định chắc xuống sàn nhà. Bàn kéo nắn cần có các mấu ngang để mắc các đai đối lực khi kéo nắn.\n- Đai đối lực: bằng vải mềm, dai, to bản (như kiểu quai ba lô) để tránh gây tổn thương cho da khi kéo nắn.\n- Thuốc tê hoặc thuốc mê: số lượng tùy thuộc người bệnh là trẻ em hay người lớn, trọng lượng người bệnh. Kèm theo là dụng cụ gây tê, gây mê, hồi sức (bơm tiêm, cồn 70o, thuốc chống sốc, mặt nạ bóp bóng, đèn nội khí quản...).\n- Bột thạch cao chuyên dụng: với người lớn cần 2-3 cuộn cỡ 15 cm và 3-4 cuộn cỡ 10 cm là đủ (kể cả 1 phần trong đó dùng để rải thành nẹp bột, trẻ em dùng ít hơn tùy theo tuổi). Ở những tuyến có điều kiện, tốt nhất là dùng loại bột đóng gói sẵn. Ở những nơi không có điều kiện, sử dụng bột tự sản xuất cũng tốt, với điều kiện bột sản xuất ra không để quá lâu, bột sản xuất quá lâu sẽ bị bão hòa hơi nước, không đảm bảo độ vững chắc nữa, bột bó xong sẽ nhanh rã, nhanh hỏng.\n- Giấy vệ sinh, bông cuộn hoặc bít tất vải xốp mềm để lót (jersey). Lưu ý, nếu dùng giấy vệ sinh, có thể gây dị ứng da người bệnh. Ở những nước phát triển, người ta thường dùng jersey, rất tiện lợi, vì nó có độ co giãn rất tốt, không gây chèn ép và có thể dùng cho nhiều kích cỡ chân tay to nhỏ khác nhau.\n- Dây rạch dọc (dùng cho bột cấp cứu, khi tổn thương 7 ngày trở lại): thường dùng một đoạn băng vải có độ dài vừa phải, vê săn lại để đảm bảo độ chắc là đủ, không cần dây chuyên dụng.\n- Dao hoặc cưa rung để rạch dọc bột trong trường hợp bó bột cấp cứu (tổn thương trong 7 ngày đầu). Nếu dùng dao rạch bột, dao cần sắc, nhưng không nên dùng dao mũi nhọn, đề phòng lỡ tay gây vết thương cho người bệnh (mặc dù tai biến này rất hiếm gặp). Nếu dùng cưa rung để rạch bột, cần lưu ý phải chờ cho bột khô hẳn mới làm, vì cưa rung chỉ cắt đứt các vật khô cứng, cũng chính vì thế chúng ta không lo ngại cưa rung làm rách da người bệnh, có chăng, nên cẩn thận khi cưa bột mà ngay ở dưới lưỡi cưa là các mấu xương (ví dụ các mắt cá, xương bánh chè...).\n- Nước để ngâm bột: đủ về số lượng để ngâm chìm hẳn các cuộn bột. Lưu ý, mùa lạnh phải dùng nước ấm, vì trong quá trình bột khô cứng sẽ tiêu hao một nhiệt lượng đáng kể làm nóng bột, có thể làm hạ thân nhiệt người bệnh, gây cảm lạnh. Nước sử dụng ngâm bột phải được thay thường xuyên để đảm bảo vệ sinh và tránh hiện tượng nước có quá nhiều cặn bột ảnh hưởng đến chất lượng bột.\n- 1 cuộn băng vải hoặc băng thun, để băng giữ ngoài bột, khi việc bó bột và rạch dọc bột đã hoàn thành.\n... BỘT CÁNH - CẲNG - BÀN TAY\n...\n2. Phương tiện\n- 01 bàn nắn thông thường, tốt nhất như kiểu bàn mổ (chắc, nặng, để khi kéo nắn bàn không bị chạy). Ở nơi không có điều kiện, có thể dùng bàn sắt, bàn gỗ, nhưng chân bàn phải được cố định chắc xuống sàn nhà. Bàn kéo nắn cần có các mấu ngang để mắc các đai đối lực khi kéo nắn.\n- Đai đối lực: bằng vải mềm, dai, to bản (như kiểu quai ba lô) để tránh gây tổn thương cho da khi kéo nắn.\n- Thuốc tê hoặc thuốc mê: số lượng tùy thuộc người bệnh là trẻ em hay người lớn, trọng lượng người bệnh. Kèm theo là dụng cụ gây tê, gây mê, hồi sức (bơm tiêm, cồn 70o, thuốc chống sốc, mặt nạ bóp bóng, đèn nội khí quản...).\n- Bột thạch cao chuyên dụng: với người lớn cần 2-3 cuộn cỡ 15 cm và 3-4 cuộn cỡ 10 cm là đủ (kể cả 1 phần trong đó dùng để rải thành nẹp bột, trẻ em dùng ít hơn tùy theo tuổi). Ở những tuyến có điều kiện, tốt nhất là dùng loại bột đóng gói sẵn. Ở những nơi không có điều kiện, sử dụng bột tự sản xuất cũng tốt, với điều kiện bột sản xuất ra không để quá lâu, bột sản xuất quá lâu sẽ bị bão hòa hơi nước, không đảm bảo độ vững chắc nữa, bột bó xong sẽ nhanh rã, nhanh hỏng.\n- Giấy vệ sinh, bông cuộn hoặc bít tất vải xốp mềm để lót (jersey). Lưu ý, nếu dùng giấy vệ sinh, có thể gây dị ứng da người bệnh. Ở những nước phát triển, người ta thường dùng jersey, rất tiện lợi, vì nó có độ co giãn rất tốt, không gây chèn ép và có thể dùng cho nhiều kích cỡ chân tay to nhỏ khác nhau.\n- Dây rạch dọc (dùng cho bột cấp cứu, khi tổn thương 7 ngày trở lại): thường dùng một đoạn băng vải có độ dài vừa phải, vê săn lại để đảm bảo độ chắc là đủ, không cần dây chuyên dụng.\n- Dao hoặc cưa rung để rạch dọc bột trong trường hợp bó bột cấp cứu (tổn thương trong 7 ngày đầu). Nếu dùng dao rạch bột, dao cần sắc, nhưng không nên dùng dao mũi nhọn, đề phòng lỡ tay gây vết thương cho người bệnh (mặc dù tai biến này rất hiếm gặp). Nếu dùng cưa rung để rạch bột, cần lưu ý phải chờ cho bột khô hẳn mới làm, vì cưa rung chỉ cắt đứt các vật khô cứng, cũng chính vì thế chúng ta không lo ngại cưa rung làm rách da người bệnh, có chăng, nên cẩn thận khi cưa bột mà ngay ở dưới lưỡi cưa là các mấu xương (ví dụ các mắt cá, xương bánh chè...).\n- Nước để ngâm bột: đủ về số lượng để ngâm chìm hẳn các cuộn bột. Lưu ý, mùa lạnh phải dùng nước ấm, vì trong quá trình bột khô cứng sẽ tiêu hao một nhiệt lượng đáng kể làm nóng bột, có thể làm hạ thân nhiệt người bệnh, gây cảm lạnh. Nước sử dụng ngâm bột phải được thay thường xuyên để đảm bảo vệ sinh và tránh hiện tượng nước có quá nhiều cặn bột ảnh hưởng đến chất lượng bột.\n- 1 cuộn băng vải hoặc băng thun, để băng giữ ngoài bột, khi việc bó bột và rạch dọc bột đã hoàn thành.\n..."""

# Parameters
max_seq_length = 1000
overlap = 128

# Tokenize the text
tokens = tokenizer.tokenize(text)

# Chunking logic
chunks = []
start = 0

while start < len(tokens):
    end = min(start + max_seq_length, len(tokens))
    chunks.append(tokens[start:end])
    start += max_seq_length - overlap  # Move to the next chunk with overlap

# Optional: Convert chunks back to text
chunk_texts = [tokenizer.convert_tokens_to_string(chunk) for chunk in chunks]

# Output the chunks
for i, chunk in enumerate(chunk_texts):
    print(f"Chunk {i+1}:\n{preprocess_text(chunk)}\n{'-'*50}")


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Chunk 1:
BỘT CÁNH CẲNG BÀN TAY ... Phương tiện 01 bàn nắn thông thường, tốt nhất như kiểu bàn mổ (chắc, nặng, để khi kéo nắn bàn không bị chạy). Ở nơi không có điều kiện, có thể dùng bàn sắt, bàn gỗ, nhưng chân bàn phải được cố định chắc xuống sàn nhà. Bàn kéo nắn cần có các mấu ngang để mắc các đai đối lực khi kéo nắn. Đai đối lực. bằng vải mềm, dai, to bản (như kiểu quai ba lô) để tránh gây tổn thương cho da khi kéo nắn. Thuốc tê hoặc thuốc mê. số lượng tùy thuộc người bệnh là trẻ em hay người lớn, trọng lượng người bệnh. Kèm theo là dụng cụ gây tê, gây mê, hồi sức (bơm tiêm, cồn 70o, thuốc chống sốc, mặt nạ bóp bóng, đèn nội khí quản...). Bột thạch cao chuyên dụng. với người lớn cần 23 cuộn cỡ 15 cm và 34 cuộn cỡ 10 cm là đủ (kể cả 1 phần trong đó dùng để rải thành nẹp bột, trẻ em dùng ít hơn tùy theo tuổi). Ở những tuyến có điều kiện, tốt nhất là dùng loại bột đóng gói sẵn. Ở những nơi không có điều kiện, sử dụng bột tự sản xuất cũng tốt, với điều kiện bột sản xuất ra không để quá 

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
import torch

In [ ]:
torch.cuda.empty_cache()
query = "Thực phẩm ăn nhẹ phù hợp trước khi chạy bộ"
query_embedding = model.encode(query)
hits = util.semantic_search(query_embedding, embeddings, top_k=50)[0]
print(query)
print("")
print("context: ")
print("")
for hit in hits[:10]:
    score = hit['score']
    corpus_id = hit['corpus_id']
    print("bi_score: ", score)

    # print(str(cids[corpus_id].decode()))
    # print(corpus.loc[(corpus['cid']) == (str(cids[corpus_id].decode())), 'text'].iloc[0])
    print(corpus.iloc[corpus_id]['text'])
    print(" ")

cross_inp = [[preprocess_text(query), preprocess_text(corpus.iloc[hit['corpus_id']]['text'])] for hit in hits]

cross_scores = cross_encoder.predict(cross_inp)
for idx in range(len(hits)):
    hits[idx]['cross-score'] = cross_scores[idx]
hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
for hit in hits[:10]:
    score = hit['cross-score']
    corpus_id = hit['corpus_id']
    print("cross_score: ", score)

    # print(str(cids[corpus_id].decode()))
    # print(corpus.loc[(corpus['cid']) == (str(cids[corpus_id].decode())), 'text'].iloc[0])
    print(corpus.iloc[corpus_id]['text'])
    print(" ")

torch.cuda.empty_cache()

In [21]:
!pip install pyngrok
!ngrok config add-authtoken 2qORvsrf7DPupP5SxpxroXZcHrt_5wvaAdPmdAyqb7JHYYN21
!pip install flask-cors

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [22]:
def cross_encoder_retrieval(cross_inp, hits):
    cross_scores = cross_encoder.predict(cross_inp)
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[:10]:
        score = hit['cross-score']
        corpus_id = hit['corpus_id']
        print("cross_score: ", score)

        print(corpus.iloc[corpus_id]['cid'])
        print(corpus.iloc[corpus_id]['text'])
        print(" ")
    best_hit = hits[0]
    return corpus.iloc[best_hit['corpus_id']]['text']

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_cors import CORS
import requests  # Thư viện để gọi API HTTP

app = Flask(__name__)
CORS(app)

# URL server ngrok thay vì sử dụng hàm `phogpt`
NGROK_SERVER_URL = "https://8250-115-78-13-71.ngrok-free.app"  # Thay bằng URL ngrok của bạn

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Lấy dữ liệu từ request.form
        question = request.form.get('message')  # 'message' khớp với key trong JavaScript
        if not question:
            return jsonify({"error": "Missing question"}), 400

        query_embedding = model.encode(question)
        hits = util.semantic_search(query_embedding, embeddings, top_k=50)[0]
        cross_inp = [[preprocess_text(question), preprocess_text(corpus.iloc[hit['corpus_id']]['text'])] for hit in hits]
        cross_scores = cross_encoder.predict(cross_inp)
        for idx in range(len(hits)):
            hits[idx]['cross-score'] = cross_scores[idx]
        hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
        best_hit = hits[0]
        top_k_cross = corpus.iloc[best_hit['corpus_id']]['text']
        print('cross-encoder')
        
        # Cross-encoder refinement
        print(top_k_cross)
        # Gọi tới server ngrok thay vì dùng phogpt
        try:
            response = requests.post(
                f"{NGROK_SERVER_URL}/ask",  # Đường dẫn tới endpoint xử lý trên server ngrok
                json={
                    "question": question,
                    "context": top_k_cross  # Gửi thông tin cần thiết
                }
            )
            response_data = response.json()

            # Kiểm tra phản hồi từ server
            if response.status_code != 200 or "answer" not in response_data:
                return jsonify({"answer": "Error processing the request to the external server."})
            
            answer = response_data["answer"]
        except Exception as e:
            print(f"Error calling external server: {str(e)}")
            return jsonify({"answer": "Error contacting external server."})

        if not answer:
            return jsonify({"answer": "Sorry, I couldn't generate an appropriate response."})
        return jsonify({"answer": answer})

    except Exception as e:
        # Log lỗi và trả về phản hồi JSON lỗi
        print(f"Error: {str(e)}")
        return jsonify({"error": f"An error occurred: {str(e)}"}), 500

# Start the Flask server
if __name__ == '__main__':
    public_url = ngrok.connect(5000)  # Kết nối Flask server với ngrok
    print(f"Public URL: {public_url}")
    app.run(host='0.0.0.0', port=5000)


Public URL: NgrokTunnel: "https://6b0c-35-189-4-26.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
# torch.cuda.empty_cache()
# query = "trốn nghĩa vụ quân sự có đi tù không ?"
# query_embedding = model.encode(query)
# hits = util.semantic_search(query_embedding, embeddings, top_k=100)[0]
# for hit in hits[:10]:
#     score = hit['score']
#     corpus_id = hit['corpus_id']
#     print("bi_score: ", score)

#     print(str(cids[corpus_id].decode()))
#     # print(corpus.loc[(corpus['cid']) == (str(cids[corpus_id].decode())), 'text'].iloc[0])
#     print(corpus.iloc[corpus_id]['text'])
#     print(" ")

# cross_inp = [[preprocess_text(query), preprocess_text(corpus.iloc[hit['corpus_id']]['text'])] for hit in hits]

# cross_scores = cross_encoder_og.predict(cross_inp)
# for idx in range(len(hits)):
#     hits[idx]['cross-score'] = cross_scores[idx]
# hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
# for hit in hits[:10]:
#     score = hit['cross-score']
#     corpus_id = hit['corpus_id']
#     print("cross_score: ", score)

#     print(str(cids[corpus_id].decode()))
#     # print(corpus.loc[(corpus['cid']) == (str(cids[corpus_id].decode())), 'text'].iloc[0])
#     print(corpus.iloc[corpus_id]['text'])
#     print(" ")

# torch.cuda.empty_cache()

In [ ]:
# import random
# from sentence_transformers import SentenceTransformer
# from sentence_transformers.evaluation import InformationRetrievalEvaluator
# from datasets import load_dataset

In [ ]:
# corpus_eval = load_dataset("hiieu/legal_eval", split="corpus")
# queries_eval = load_dataset("hiieu/legal_eval", split="queries")
# relevant_docs_data = load_dataset("hiieu/legal_eval_label", split="train")
# # Convert the datasets to dictionaries
# corpus_eval = dict(zip(corpus_eval["id"], corpus_eval["text"]))  # Our corpus (cid => document)
# queries_eval = dict(zip(queries_eval["id"], queries_eval["text"]))  # Our queries (qid => question)
# relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
# for qid, corpus_ids in zip(relevant_docs_data["question_id"], relevant_docs_data["corpus_id"]):
#     qid = str(qid)
#     corpus_ids = str(corpus_ids)
#     if qid not in relevant_docs:
#         relevant_docs[qid] = set()
#     relevant_docs[qid].add(corpus_ids)

In [ ]:
# import json
# import torch
# from sentence_transformers import SentenceTransformer
# from sentence_transformers.evaluation import (
#     InformationRetrievalEvaluator,
#     SequentialEvaluator,
# )
# from sentence_transformers.util import cos_sim
# from datasets import load_dataset, concatenate_datasets

# matryoshka_dimensions = [768] # Important: large to small
# matryoshka_evaluators = []
# # Iterate over the different dimensions
# for dim in matryoshka_dimensions:
#     ir_evaluator = InformationRetrievalEvaluator(
#         queries=queries_eval,
#         corpus=corpus_eval,
#         relevant_docs=relevant_docs,
#         name=f"dim_{dim}",
#         truncate_dim=dim,  # Truncate the embeddings to a certain dimension
#         score_functions={"cosine": cos_sim},
#     )
#     matryoshka_evaluators.append(ir_evaluator)

# # Create a sequential evaluator
# evaluator = SequentialEvaluator(matryoshka_evaluators)

In [ ]:
# # Evaluate the model
# results = evaluator(model)
# for k,v in results.items():
#     print(k, v)

In [ ]:
# model_m3 = SentenceTransformer("/kaggle/input/bge_m3_finetuned/transformers/default/1/checkpoint-5000")

In [ ]:
# results = evaluator(model)
# for k,v in results.items():
#     print(k, v)

In [ ]:
# import pandas as pd
# import nltk
# from nltk.tokenize import word_tokenize

# # Download the punkt tokenizer if not already downloaded
# nltk.download('punkt')

# # Load the CSV file into a DataFrame
# df = pd.read_csv('/kaggle/input/corpus-after-preprocessed-18de/corpus_after_preprocessed_18Dec.csv')  # Replace 'your_file.csv' with your actual file path

# # Specify the column name containing the text data
# text_column = 'text'  # Replace 'your_column_name' with the actual column name

# # Ensure the specified column exists in the DataFrame
# if text_column not in df.columns:
#     raise ValueError(f"Column '{text_column}' not found in the CSV file.")

# # Tokenize the text in each row of the specified column and count the tokens
# token_counts = df[text_column].dropna().apply(lambda text: len(word_tokenize(text)))

# # Calculate the maximum and mean token counts
# max_tokens = token_counts.max()
# mean_tokens = token_counts.mean()

# # Print the results
# print(f"Maximum number of tokens in a single entry: {max_tokens}")
# print(f"Mean number of tokens across all entries: {mean_tokens:.2f}")


# evaluate

In [ ]:
# import pandas as pd
# import numpy as np

# # Đọc file predict.zip

# def read_predict_file(file_path):
#     predictions = {}
#     with open(file_path, 'r') as f:
#         for line in f:
#             parts = line.strip().split()
#             qid = parts[0]
#             cids = [cid.split('_')[0] for cid in parts[1:]]
#             predictions[qid] = cids
#     return predictions

# def parse_context(text):
#     result = []
#     parts = text.strip("[]").split("\n")
#     for part in parts:
#         part = part.strip()
#         if "'" in part:
#             result.append(part.strip("'"))
#         else:
#             result.extend(part.split())
#     return result

# # Đọc file CSV gốc (9456 sample cuối)
# def read_ground_truth(train):
#     # Lấy 9456 sample cuối
#     df = train.head(9456)
    
#     # Tạo từ điển để tra cứu nhanh
#     ground_truth = {}
#     for _, row in df.iterrows():
#         ground_truth[str(row['qid'])] = row['cid']
    
#     return ground_truth

# # Tính MRR@10
# def calculate_mrr(predictions, ground_truth):
#     mrr_scores = []
#     for qid, label_list in ground_truth.items():
#         if qid in predictions:
#             predicted_cids = [str(cid) for cid in predictions[qid]]
#             rank = None
#             for i, cid in enumerate(predicted_cids):
#                 if cid in [str(label) for label in label_list]:
#                     rank = i + 1
#                     break
#             if rank is not None:
#                 mrr_scores.append(1 / rank)
#             else:
#                 mrr_scores.append(0)
#         else:
#             mrr_scores.append(0)
    
#     return np.mean(mrr_scores)

# # Đường dẫn file
# predict_file = '/kaggle/input/query-ela-jin/predict.zip'  # Hoặc 'predict.txt' tùy cách bạn lưu
# csv_file = '/kaggle/input/query-ela-jin/train_csv.csv'

In [ ]:
# import py_vncorenlp
# rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/input/vn-core-nlp/VnCoreNLP-master')

In [ ]:
# train = pd.read_csv("/kaggle/input/train-after-preprocessed-18dec/train_after_preprocessed_18Dec.csv")
# train['cid'] = train['cid'].apply(lambda x: x.strip("[]"))
# train['cid'] = train['cid'].astype(str)
# train_subset = train.iloc[90000:95000]

In [ ]:
# train_subset.info()

In [ ]:
# from tqdm import tqdm

In [ ]:
# count = 0
# total_queries = len(train_subset)
# for test_case in tqdm(range(total_queries)):
#     query = train_subset.iloc[test_case]['question']
#     print(query)
#     if count == 10:
#         break
#     count += 1

In [ ]:
# def calculate_mrr_at_k(test_data, k=50):
#     """
#     Calculate MRR@K based on the re-ranked document IDs.

#     Parameters:
#         test_data (list): List of test cases in the format [{'query': str, 'cid': int, 'top_k_cross': list}, ...].
#                          Each `top_k_cross` is a list of cids (document IDs) ranked by the cross-encoder.
#         k (int): Number of top documents to consider for MRR@K.

#     Returns:
#         float: MRR@K score for the test dataset.
#     """
#     mrr_score = 0.0
#     total_queries = len(test_data)

#     for test_case in tqdm(range(total_queries)):
#         query = test_data.iloc[test_case]['question']
#         top_k = 50
#         query_embedding = model.encode(query)
        
#         hits = util.semantic_search(query_embedding, embeddings, top_k=top_k)[0]

#         top_k_bi = []
#         for hit in hits:
#             top_k_bi.append(str(cids[hit['corpus_id']].decode()))
        
#         # cross_inp = [[query, preprocess_text(corpus.iloc[hit['corpus_id']]['context'])] for hit in hits]
#         # cross_scores = cross_encoder.predict(cross_inp)
#         # for idx in range(len(hits)):
#         #     hits[idx]['cross-score'] = cross_scores[idx]
#         # hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
#         # top_k_cross = cids[corpus_id].decode()

#         if len(test_data.iloc[test_case]['cid']) > 1:
#             true_cid = test_data.iloc[test_case]['cid']
        
#         # Calculate Reciprocal Rank (RR) for this query
#         rr = 0.0
#         for rank, cid in enumerate(top_k_bi, start=1):
#             if cid == true_cid:
#                 rr = 1 / rank
#                 break

#         mrr_score += rr

#     # Compute the Mean Reciprocal Rank (MRR@K)
#     mrr_score /= total_queries
#     return mrr_score

In [ ]:
# bi_score = calculate_mrr_at_k(train_subset)

In [ ]:
# print("halong_embedding_finetuned")
# print(f"MRR@10: {bi_score}")

In [ ]:
# bi_score = calculate_mrr_at_k(train_subset)
# print("halong_embedding_finetuned")
# print(f"MRR@10: {bi_score}")

In [ ]:
# print("halong_embedding_finetuned")
# print(f"MRR@50: {bi_score}")